In [ ]:
!pip install fastf1 plotly matplotlib pandas numpy pyarrow

In [ ]:
#imports

import fastf1, pandas as pd, numpy as np, matplotlib.pyplot as plt, plotly.express as px, pyarrow
print("FastF1:", fastf1.__version__)

In [ ]:
from pathlib import Path

PROJECT_PATH = Path("..").resolve()

cache_dir = PROJECT_PATH / "f1cache"
cache_dir.mkdir(exist_ok=True)

import fastf1
fastf1.Cache.enable_cache(cache_dir)

print("Cache set to:", cache_dir)


In [ ]:
#Race Setup
session = fastf1.get_session(2024, "Monza", "R")
session.load()
print(session)

#Driver Setup - Max Verstappen and Lewis Hamilton
ver_lap = session.laps.pick_driver("VER").pick_fastest()
ham_lap = session.laps.pick_driver("HAM").pick_fastest()
print(ver_lap)
print(ham_lap)



In [ ]:
#obtaining telemetry data for each driver

ver_tel = ver_lap.get_car_data().add_distance()
ham_tel = ham_lap.get_car_data().add_distance()

print(ver_tel)
print(ham_tel)

In [ ]:
ver_tel['Driver'] = 'Ver'
ham_tel['Driver'] = 'Ham'

In [ ]:
both_tel = pd.concat([ver_tel,ham_tel])
print(both_tel)

In [ ]:
dmin = max(ver_tel['Distance'].min(), ham_tel['Distance'].min())
dmax = min(ver_tel['Distance'].max(), ham_tel['Distance'].max())
distance = np.linspace(dmin, dmax, 2000)

def interp_col(df, col):
    return np.interp(distance, df['Distance'], df[col])

merged = pd.DataFrame({
    "Distance": distance,
    "VER_Speed": interp_col(ver_tel, "Speed"),
    "HAM_Speed": interp_col(ham_tel, "Speed"),
    "VER_Throttle": interp_col(ver_tel, "Throttle"),
    "HAM_Throttle": interp_col(ham_tel, "Throttle"),
    "VER_Brake": interp_col(ver_tel, "Brake"),
    "HAM_Brake": interp_col(ham_tel, "Brake"),
    "VER_nGear": interp_col(ver_tel, "nGear"),
    "HAM_nGear": interp_col(ham_tel, "nGear"),
})


In [ ]:
#speed vs distance plot

plt.figure(figsize=(10,5))

plt.plot(merged['Distance'], merged["VER_Speed"], label="Verstappen", color="blue")
plt.plot(merged['Distance'], merged["HAM_Speed"], label="Hamilton", color="red")

plt.xlabel("Distance (m)")
plt.ylabel("Speed (km/h)")
plt.title("Speed vs Distance (fastest lap)")
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(PROJECT_PATH, "output", "speed_vs_distance.jpg"), dpi=150)

plt.show()


In [ ]:
#brake vs throttle plot

plt.figure(figsize=[10,5])


plt.plot(merged['VER_Brake'], merged["VER_Throttle"], label="Verstappen", color="blue")
plt.plot(merged['HAM_Brake'], merged["HAM_Throttle"], label="Hamilton", color="red")

plt.xlabel("Brake")
plt.ylabel("Throttle")
plt.title("Brake vs Throttle")
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(PROJECT_PATH, "output", "brake_vs_throttle.jpg"), dpi=150)

plt.show()


In [ ]:
#delta time array

v_ver = merged["VER_Speed"] * 1000 / 3600
v_ham = merged["HAM_Speed"] * 1000 / 3600

ds = np.gradient(merged["Distance"].values)

tiny = 1e-6
dt_ver = ds / np.clip(v_ver, tiny, None)
dt_ham = ds / np.clip(v_ham, tiny, None)

t_ver = np.cumsum(dt_ver)
t_ham = np.cumsum(dt_ham)

t_ver = t_ver - t_ver[0]
t_ham = t_ham - t_ham[0]

merged["VER_Time"] = t_ver
merged["HAM_Time"] = t_ham
merged["DeltaTime"] = t_ver - t_ham


In [ ]:
#distance vs delta time
import os
plt.figure(figsize=[10,5])

plt.plot(merged['Distance'], merged["VER_Time"], label="Verstappen", color="blue")
plt.plot(merged['Distance'], merged["HAM_Time"], label="Hamilton", color="red")

plt.xlabel("Distance")
plt.ylabel("Delta_Time")
plt.title("Distance vs Delta-time")
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(PROJECT_PATH, "output", "distance_vs_delta_time.jpg"), dpi=150)

plt.show()

In [ ]:
ver_geo = ver_lap.get_telemetry().add_distance()
ham_geo = ham_lap.get_telemetry().add_distance()


print(ver_geo.columns)


In [ ]:
ver_geo["X_flip"] = ver_geo["Y"]
ver_geo["Y_flip"] = ver_geo["X"]

ham_geo["X_flip"] = ham_geo["Y"]
ham_geo["Y_flip"] = ham_geo["X"]


fig_ver_flip = px.scatter(
    ver_geo,
    x="X_flip", y="Y_flip",
    color="Speed",
    title="Verstappen Racing Line (Flipped Axes, Colored by Speed)",
    labels={"X_flip": "Flipped X", "Y_flip": "Flipped Y", "Speed": "Speed (km/h)"}
)
fig_ver_flip.update_yaxes(scaleanchor="x", scaleratio=1)
fig_ver_flip.update_traces(mode="markers", marker={"size": 3})
fig_ver_flip.write_html(os.path.join(PROJECT_PATH, "output", "map_VER_flipped.html"))

fig_ham_flip = px.scatter(
    ham_geo,
    x="X_flip", y="Y_flip",
    color="Speed",
    title="Hamilton Racing Line (Flipped Axes, Colored by Speed)",
    labels={"X_flip": "Flipped X", "Y_flip": "Flipped Y", "Speed": "Speed (km/h)"}
)
fig_ham_flip.update_yaxes(scaleanchor="x", scaleratio=1)  # keep proportions
fig_ham_flip.update_traces(mode="markers", marker={"size": 3})
fig_ham_flip.write_html(os.path.join(PROJECT_PATH, "output", "map_HAM_flipped.html"))

fig_ver_flip.show()
fig_ham_flip.show()


In [ ]:
import plotly.express as px

gear_df = pd.DataFrame({
    "Distance": merged["Distance"].tolist() * 2,
    "Gear": pd.concat([merged["VER_nGear"], merged["HAM_nGear"]], ignore_index=True),
    "Driver": ["VER"] * len(merged) + ["HAM"] * len(merged)
})

fig_gear = px.scatter(
    gear_df,
    x="Distance", y="Gear",
    color="Driver",
    title="Gear Usage vs Distance (Fastest Lap)",
    labels={"Distance": "Distance (m)", "Gear": "Gear"},
    opacity=0.6
)

fig_gear.write_html(os.path.join(PROJECT_PATH, "output", "gear_usage.html"))

fig_gear.show()
